# 100daysofML

## Titanic: Machine Learning from Disaster 

### Resumo


teste teste teste

teste test 2 

### Importanto bibliotecas utilizadas 

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm as cm #colormap
import seaborn as sns

% matplotlib inline


### Abrindo dataset 

In [2]:
#Abrindo dataset com ID passagens e sobrevivencia, onde 1 = sobreviveu e 0 = não sobreviveu
#gender_submission = pd.read_csv("gender_submission.csv")

#Se eu quero filtrar uma coluna, devo colocar como a coluna index e depois filtrar usando a função filter do pandas como abaixo
#gender_submission = pd.read_csv("gender_submission.csv", index_col = "Survived")
#gender_submission.filter(like='0', axis=0)
        
#Abrindo do dataset de treino do modelo que contem PassengerId, Survived, Pclass, Name, Sex, Age, SibSp, Parch, Ticket, Fare, Cabin, Embarked
treino = pd.read_csv("train.csv")
teste = pd.read_csv("test.csv")

treino.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


## Informações dos datasets

In [3]:
treino.info()
print("----------------------------------")
teste.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
PassengerId    891 non-null int64
Survived       891 non-null int64
Pclass         891 non-null int64
Name           891 non-null object
Sex            891 non-null object
Age            714 non-null float64
SibSp          891 non-null int64
Parch          891 non-null int64
Ticket         891 non-null object
Fare           891 non-null float64
Cabin          204 non-null object
Embarked       889 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 83.6+ KB
----------------------------------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 11 columns):
PassengerId    418 non-null int64
Pclass         418 non-null int64
Name           418 non-null object
Sex            418 non-null object
Age            332 non-null float64
SibSp          418 non-null int64
Parch          418 non-null int64
Ticket         418 non-null objec

#### Descrição do dataset 
1. Do nosso dataset de treino: 
    - 891 entradas (passageiros)
    - 12 colunas, sendo que dessas colunas 7 são numéricas


2. Do nosso dataset de teste
    - 414 (passageiros)
    - 11 colunas, sendo que 6 são numéricas

In [4]:
#Descrição das variaveis 

variaveis_info = pd.DataFrame({"Variáves": ["PassengerId", 'Survived', 'Pclass', 'Name', 'Sex', 'Age','SibSp', 'Parch', 'Ticket', 'Fare',' Cabin', 'Embarked'
], "Definição":["Identificação do passageiros", "Sobrevivência", "Classe da passagem", 'Nome', 'Sexo', 'Idade', 'Irmão/conjuges a bordo', 'dos pais/filhos a bordo', 'Numero da passagem', 'Valor da passagem', 'Numero da cabine', 'Portão de embarque'],"Chaves":[" "," 1 = Sobreviveu; 0 = Não sobreviveu", " 1 = 1st, 2 = 2nd, 3 = 3rd", " ", " ", " ", " ", " ", " ", " ", " ", "C = Cherbourg, Q = Queenstown, S = Southampton"]})
variaveis_info

,Variáves,Definição,Chaves
0,PassengerId,Identificação do passageiros,
1,Survived,Sobrevivência,1 = Sobreviveu; 0 = Não sobreviveu
2,Pclass,Classe da passagem,"1 = 1st, 2 = 2nd, 3 = 3rd"
3,Name,Nome,
4,Sex,Sexo,
5,Age,Idade,
6,SibSp,Irmão/conjuges a bordo,
7,Parch,dos pais/filhos a bordo,
8,Ticket,Numero da passagem,
9,Fare,Valor da passagem,


### Limpeza dos dados usando os 4 C's 
     - Corrigindo  - Se há outliers 
     - Complementando - Se há valores null 
     - Criando - Feature engineenging 
     - Convertendo - dados de diferente tipo (int, float, object...)

In [5]:
print('Treino com valores vazios:\n', treino.isnull().sum()) 
print("----------------------------")
print('Teste com valores vazios:\n',teste.isnull().sum())

Treino com valores vazios:
 PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64
----------------------------
Teste com valores vazios:
 PassengerId      0
Pclass           0
Name             0
Sex              0
Age             86
SibSp            0
Parch            0
Ticket           0
Fare             1
Cabin          327
Embarked         0
dtype: int64


Valores como Age, Cabin e Embarked  no dataset de treino estão vazios 

Valores como Age, Cabin e Fare no dataset de teste estão vazios

In [6]:
#Dados estatisticos do treino 
treino.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


In [7]:
#Dados estatisticos do teste 
teste.describe()

,PassengerId,Pclass,Age,SibSp,Parch,Fare
count,418.000000,418.000000,332.000000,418.000000,418.000000,417.000000
mean,1100.500000,2.265550,30.272590,0.447368,0.392344,35.627188
std,120.810458,0.841838,14.181209,0.896760,0.981429,55.907576
min,892.000000,1.000000,0.170000,0.000000,0.000000,0.000000
25%,996.250000,1.000000,21.000000,0.000000,0.000000,7.895800
50%,1100.500000,3.000000,27.000000,0.000000,0.000000,14.454200
75%,1204.750000,3.000000,39.000000,1.000000,0.000000,31.500000
max,1309.000000,3.000000,76.000000,8.000000,9.000000,512.329200


In [8]:
treino.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


### Preencher valores vazios e remover colunas que creio que não seja util  

In [9]:
limpeza_dados = [treino, teste]

for dataset in limpeza_dados:    
    #preencher com a media dos valores
    dataset['Age'].fillna(dataset['Age'].mean(), inplace = True)

    #preencher embarked com a moda
    dataset['Embarked'].fillna(dataset['Embarked'].mode()[0], inplace = True)

    #preencher fare with mediana
    dataset['Fare'].fillna(dataset['Fare'].median(), inplace = True)
    
#deletando colunas de passengerId, Cabin e Ticket 
drop_column = ['PassengerId','Cabin', 'Ticket']
treino.drop(drop_column, axis = 1, inplace = True)

print(treino.isnull().sum()) 
print("----------------------------")
print(teste.isnull().sum())


Survived    0
Pclass      0
Name        0
Sex         0
Age         0
SibSp       0
Parch       0
Fare        0
Embarked    0
dtype: int64
----------------------------
PassengerId      0
Pclass           0
Name             0
Sex              0
Age              0
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          327
Embarked         0
dtype: int64


### Feature Engineering 


In [25]:
#Apartir das Features SibSp e Parch, foi criado uma nova feature do tamanho da familia  
for dataset in limpeza_dados:
    dataset["FamilySize"] = dataset["SibSp"] + dataset["Parch"] + 1
    
    #Verificar se é um passageiro sozinho ou com familia 
    dataset["Alone"] = 1 #onde 1 = sim 
    dataset["Alone"].loc[dataset["FamilySize"] > 1] = 0 # onde 0 = não 
    
    dataset["Title"] = dataset["Name"].str.split(", ", expand = True)[1].str.split(".", expand = True)[0] #separando o titulo Mrs, miss, mister, etc e criando ua nova coluna 

#Aqui eu transformo os valores de titulo em true ou false e filtro tudo que sua soma for menos de 10 fica em uma categoria chamada "Other"
nomes_titulo = (treino["Title"].value_counts() < 10)
treino['Title'] = treino['Title'].apply(lambda x: 'Other' if nomes_titulo.loc[x] == True else x)

print(treino['Title'].value_counts())
print("-"*10) 


treino.info()
teste.info()
treino.sample(10)

Mr        517
Miss      182
Mrs       125
Master     40
Other      27
Name: Title, dtype: int64
----------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 15 columns):
Survived           891 non-null int64
Pclass             891 non-null int64
Name               891 non-null object
Sex                891 non-null object
Age                891 non-null float64
SibSp              891 non-null int64
Parch              891 non-null int64
Fare               891 non-null float64
Embarked           891 non-null object
FamilySize         891 non-null int64
Alone              891 non-null int64
Title              891 non-null object
Sex_Encode         891 non-null int64
Embarked_Encode    891 non-null int64
Title_Encode       891 non-null int64
dtypes: float64(2), int64(9), object(4)
memory usage: 104.5+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 17 columns):
PassengerId        418 non-null int64
Pcl

C:\anaconda3\lib\site-packages\pandas\core\indexing.py:189: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Fare,Embarked,FamilySize,Alone,Title,Sex_Encode,Embarked_Encode,Title_Encode
199,0,2,"Yrois, Miss. Henriette (""Mrs Harbeck"")",female,24.00,0,0,13.0000,S,1,1,Miss,0,2,8
439,0,2,"Kvillner, Mr. Johan Henrik Johannesson",male,31.00,0,0,10.5000,S,1,1,Mr,1,2,11
856,1,1,"Wick, Mrs. George Dennick (Mary Hitchcock)",female,45.00,1,1,164.8667,S,3,0,Mrs,0,2,12
794,0,3,"Dantcheff, Mr. Ristiu",male,25.00,0,0,7.8958,S,1,1,Mr,1,2,11
329,1,1,"Hippach, Miss. Jean Gertrude",female,16.00,0,1,57.9792,C,2,0,Miss,0,0,8
857,1,1,"Daly, Mr. Peter Denis",male,51.00,0,0,26.5500,S,1,1,Mr,1,2,11
875,1,3,"Najib, Miss. Adele Kiamie ""Jane""",female,15.00,0,0,7.2250,C,1,1,Miss,0,0,8
795,0,2,"Otter, Mr. Richard",male,39.00,0,0,13.0000,S,1,1,Mr,1,2,11
52,1,1,"Harper, Mrs. Henry Sleeper (Myna Haxtun)",female,49.00,1,0,76.7292,C,2,0,Mrs,0,0,12
644,1,3,"Baclini, Miss. Eugenie",female,0.75,2,1,19.2583,C,4,0,Miss,0,0,8


## Preparação para modelo

### Converter valores categorico

In [28]:
from sklearn import preprocessing

label = preprocessing.LabelEncoder()

for dataset in limpeza_dados:
    dataset["Sex_Encode"] = label.fit_transform(dataset["Sex"])
    dataset["Embarked_Encode"] = label.fit_transform(dataset["Embarked"])
    dataset["Title_Encode"] = label.fit_transform(dataset["Title"])
    
#Essa será nossa varialvel alvo, ou seja, varialve y 
Alvo = ["Survived"]

#Variaveis x
treino_x = ['Sex','Pclass', 'Embarked', 'Title','SibSp', 'Parch', 'Age', 'Fare', 'FamilySize', 'Alone']
treino_x_calc = ["Sex_Encode", "Pclass", "Embarked_Encode", "Title_Encode", "SibSp", "Parch", "Age", "Fare"]

treino_xy = Alvo + treino_x

print("Original X Y: " , treino_xy, "\n")

treino_dummy = pd.get_dummies(treino[treino_x])
treino_x_dummy = treino_dummy.columns.tolist()
treino_xy_dummy = Alvo + treino_x_dummy
print('Dummy X Y: ', treino_xy_dummy, '\n')

discard_columns = ['Title_Dr', 'Title_Rev', 'confidence', 'contexDialogActionSlug', 
                   'monesConversationName', 'dialogSystem', 'monesConversationSlug', 
                   'isAnythingElse', 'slugBot','workspaceSlug', 'slugProvider',
                   'dialogName', 'dialogSlug']
treino_dummy.head()

Original X Y:  ['Survived', 'Sex', 'Pclass', 'Embarked', 'Title', 'SibSp', 'Parch', 'Age', 'Fare', 'FamilySize', 'Alone'] 

Dummy X Y:  ['Survived', 'Pclass', 'SibSp', 'Parch', 'Age', 'Fare', 'FamilySize', 'Alone', 'Sex_female', 'Sex_male', 'Embarked_C', 'Embarked_Q', 'Embarked_S', 'Title_Master', 'Title_Miss', 'Title_Mr', 'Title_Mrs', 'Title_Other'] 



,Pclass,SibSp,Parch,Age,Fare,FamilySize,Alone,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S,Title_Master,Title_Miss,Title_Mr,Title_Mrs,Title_Other
0,3,1,0,22.0,7.2500,2,0,0,1,0,0,1,0,0,1,0,0
1,1,1,0,38.0,71.2833,2,0,1,0,1,0,0,0,0,0,1,0
2,3,0,0,26.0,7.9250,1,1,1,0,0,0,1,0,1,0,0,0
3,1,1,0,35.0,53.1000,2,0,1,0,0,0,1,0,0,0,1,0
4,3,0,0,35.0,8.0500,1,1,0,1,0,0,1,0,0,1,0,0


### Verificando se os dados estão "normalizados"

In [12]:
print("Treino com valores vazios: \n", treino.isnull().sum())
print("-"*10)
print (treino.info())
print("-"*10)

print('Teste com valores vazios:\n', teste.isnull().sum())
print("-"*10)
print (teste.info())
print("-"*10)   

Treino com valores vazios: 
 Survived           0
Pclass             0
Name               0
Sex                0
Age                0
SibSp              0
Parch              0
Fare               0
Embarked           0
FamilySize         0
Alone              0
Title              0
Sex_Encode         0
Embarked_Encode    0
Title_Encode       0
dtype: int64
----------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 15 columns):
Survived           891 non-null int64
Pclass             891 non-null int64
Name               891 non-null object
Sex                891 non-null object
Age                891 non-null float64
SibSp              891 non-null int64
Parch              891 non-null int64
Fare               891 non-null float64
Embarked           891 non-null object
FamilySize         891 non-null int64
Alone              891 non-null int64
Title              891 non-null object
Sex_Encode         891 non-null int64
Embarked_Encode    891 no

### Separar dataset  treino e teste

* Essa etapa é importante para evitar **overfit**

In [32]:
from sklearn import model_selection

#esse split pode ser feito manualmente ou usando o sklearn, para evitar um bias usarei o sklearn
train_x, test_x, train_y, test_y = model_selection.train_test_split(treino[treino_x_calc], treino[Alvo], random_state = 0)

train_x_dummy, test_x_dummy, train_y_dummy, test_y_dummy = model_selection.train_test_split(treino_dummy[treino_x_dummy], treino[Alvo], random_state = 0)

print("Treino tamanho: {}".format(treino.shape))
print("Train tamanho: {}".format(train_x.shape))
print("Test tamanho: {}".format(test_x.shape))


Treino tamanho: (891, 15)
Train tamanho: (668, 8)
Test tamanho: (223, 8)


In [ ]:
treino[['Sex', 'Survived']].groupby(['Sex'], as_index=False).mean().sort_values(by='Survived', ascending=False)

In [ ]:
treino[['Pclass', 'Survived']].groupby(['Pclass'], as_index=False).mean().sort_values(by='Survived', ascending=False)

g = sns.FacetGrid(treino, col='Survived')
g.map(plt.hist, 'Pclass', bins=20)

In [ ]:
#Agrupamento de sobrevivente por localização de embarque
treino[['Embarked', 'Survived']].groupby(['Embarked'], as_index=False).mean().sort_values(by='Survived', ascending=False)

In [ ]:
#histograma de sobreviventes por idade 

g = sns.FacetGrid(treino, col='Survived')
g.map(plt.hist, 'Age', bins=20)

In [ ]:
#heatmap 

fig = plt.figure(figsize = (15,8))

ax1 = fig.add_subplot(111)
cmap = cm.get_cmap('Greys', 30)
cax = ax1.imshow(treino.corr(), interpolation="nearest", cmap=cmap)
plt.title("Matrix de correlação")
labels = list(treino)
ax1.set_xticklabels(labels,fontsize=10)
ax1.set_yticklabels(labels,fontsize=10)
fig.colorbar(cax)
plt.show()


#### Conclusões tiradas com os dados estatisticos 
    - Na coluna Age, é possivel observar que há missing values e a idade média é de ~30 anos, ou seja pacientes mais novos
    - Cerca de 38% dos passageiros sobreviveram 
    - A maioria das passagens era da segunda classe
    - Passageiros da classe 3 foram os que mais morreram 
    - Mulheres sobreviveram mais que homens 
    

In [ ]:
#Dados das colunas de strings 
treino.describe(include=['O'])


#### Conclusões tiradas 
    - As colunas Cabin e Embarked há missing values 
    - Cerca de 65% dos passageiros são do sexo masculino 
    - Apesar de não possuir 3 valores é possivel falar que a maioria dos passageiros vem de Southampton
    - Alguns passageiros dividem cabines, poso dizer isso pois há valores duplicados
    - há certa de 23% de tickets duplicados
   

### Decisões que podem ser tomadas 
    - Considerar Classe, Idade, Sexo para o modelo 
    
#### Então vamos explorar mais esses dados

In [ ]:
#Analisando Classe juntamente com sobrevivencia e idade, pois observei que há uma variação entre essas variaveis  

grid = sns.FacetGrid(treino, col='Survived', row='Pclass', size=2.2, aspect=1.6)
grid.map(plt.hist, 'Age', alpha=.5, bins=20)
grid.add_legend();

### Conclusões tiradas 
    - Na Classe 1 
        - como já pontuei anteriormente passageiros dessa classe sobreviveram mais que morreram 
        - a distribuição de idade é abrangente, então não houve nessa classe em especifico uma prioridade entre as idades dos passageiros 
    - Na Classe 2 
        - Nessa classe passageiros entre 20 e 40 anos foram os que mais morreram 
    - Na Classe 3
        - Foi a classe que mais tinha passageiros, e que mais morreram sendo a idade entre 20 e 40 anos predominante 
        - Aqui podemos observar que passageiros jovens tiveram prioridade em que quesito de sobrevivencia    

In [ ]:
# Agora vamos explorar Sexo, Embarque, classe e sobrevivencia 

grid = sns.FacetGrid(treino, row='Embarked', size=2.2, aspect=1.6)
grid.map(sns.pointplot, 'Pclass', 'Survived', 'Sex', palette='deep')
grid.add_legend()
